In [89]:
import numpy as np
import pandas as pd
from math import exp
import copy
from sklearn import preprocessing

In [90]:
# Feed Forward helper methods
def sigmoid(value):
    return 1.0/(1+exp(value * (-1)))

def sigma(matrix_weight, matrix_input, bias=0):
    # Prereq: len(arr_weight) = len(arr_input)
    return matrix_weight.dot(matrix_input.transpose()) + bias


# hidden_layer = int (number of hidden layers)
# nb_nodes = arr[int] (number of nodes per hidden layer)
# len_input_matrix = int (number of features)
# Output: List of Matrixes
# Method: He initialization
# Link: https://towardsdatascience.com/weight-initialization-techniques-in-neural-networks-26c649eb3b78
def initialize_weights(hidden_layer, nb_nodes, len_input_matrix):
    # nb_nodes = nb_nodes.astype(int) #dicek nanti perlu atau ga
    arr_weight_this_batch = list()
    for i in range(hidden_layer):
        if i==0:
            nb_nodes_prev = len_input_matrix
        else:
            nb_nodes_prev = nb_nodes[i-1]
        weight_matrix = np.random.randn(nb_nodes[i], nb_nodes_prev) * np.sqrt(2/(nb_nodes_prev+nb_nodes[i]))
        arr_weight_this_batch.append(weight_matrix)
    
    return arr_weight_this_batch

In [91]:
# Backpropagation and Update Weight helper methods
def error(feed_forward_output, target_output):
    return 0.5*((target_output-feed_forward_output)**2)

def propagate_error_output_layer(feed_forward_output, target_output):
    return feed_forward_output*(1-feed_forward_output)*(target_output-feed_forward_output)

def propagate_error_hidden_layer_neuron(arr_weight_input, arr_neuron_input, arr_weight_output, arr_neuron_output):
    #Input here means input in a single neuron, while output means output of a single neuron
    #len(arr_weight) = len(arr_input)
    sigma_input, sigma_output = sigma(arr_weight_input, arr_neuron_input), sigma(arr_weight_output, arr_neuron_output)
    return sigmoid(sigma_input) * (1 - sigmoid(sigma_input)) * sigma_output

# error = neuron's error
def update_weight_neuron(weight_prev_prev, weight_prev, learning_rate, momentum, error, input_neuron):
    # weight_prev_prev = previous of weight_prev
    return weight_prev + weight_prev_prev * momentum + learning_rate*error*input_neuron

In [92]:
# input_matrix = matrix[float] (data) (asumsi, kolom terakhir adalah hasil klasifikasi)
# hidden_layers = int (number of hidden layers)
# nb_nodes = arr[int] (number of nodes per hidden layer)
# nu = float (momentum)
# alfa = float (learning rate)
# epoch = int (number of training loops)
# batch_size = int (mini-batch)
def mini_batch_gradient_descent(input_matrix, hidden_layer, nb_nodes, nu, alfa, epoch, batch_size=1):
    
    #transpose-slicing, memisah input dan label
    col_width = input_matrix.shape[1]
    input_data = (input_matrix.transpose()[0:col_width-1]).transpose()
    label_data = (input_matrix.transpose()[col_width-1:col_width]).transpose()
    #print(input_data, "\n")
    #print(label_data, "\n")
    
    hidden_layer += 1
    nb_nodes.append(1) #as total node in output layer, to simplify 
    for no_epoch in range(epoch):
        if no_epoch == 0:
            arr_weight_this_batch = initialize_weights(hidden_layer, nb_nodes, col_width-1)
        else:
            arr_weight_prev_batch = copy.deepcopy(arr_weight_this_batch) # tracking previous state of weights
            
        for no_input_data in range(len(input_data)):
            #Feed Forward
            all_sigma_values = list()
            for no_hidden_layer in range(hidden_layer):
                if no_hidden_layer == 0:
                    all_sigma_values.append(sigma(arr_weight_this_batch[no_hidden_layer], input_data[no_input_data]))
                else:
                    all_sigma_values.append(sigma(arr_weight_this_batch[no_hidden_layer], all_sigma_values[no_hidden_layer-1]))
                for no_rows in range(len(all_sigma_values[no_hidden_layer])):
                    all_sigma_values[no_hidden_layer][no_rows] = sigmoid(all_sigma_values[no_hidden_layer][no_rows])
            #Result of sigma will be array with 1 element only, so it's safe to select like this
            error_value = error(all_sigma_values[no_hidden_layer][0], label_data[no_input_data])[0]
            print("From data in epoch " + str(no_epoch) + " and no_input " + str(no_input_data) + " has error " + str(error_value))
            
            #Back Propagation
            output_error = propagate_error_output_layer(all_sigma_values[no_hidden_layer][0], label_data[no_input_data])
            #for no_hidden_layer in range(hidden_layer-1, -1, -1):
                #print(no_hidden_layer)
            #print(hidden_layer)
            #update_weight_neuron(weight_prev_prev, weight_prev, nu, alfa, error, output_neuron):
    #return output

In [93]:
# dataset load and preprocess
df = pd.read_csv("weather.csv")
#print(df.head, "\n")

# transform non-numeric data to numeric data (boolean type is already 0 and 1)
encoder = preprocessing.LabelEncoder()
df["outlook"] = encoder.fit_transform(df["outlook"])
df["play"] = encoder.fit_transform(df["play"])
#print(df.head, "\n")

#scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() #feature_range=(0,1) (min_val, max_val)
data_matrix = scaler.fit_transform(df) # di tahap ini, data sudah menjadi numpy array
#print(data_matrix, "\n")

result = mini_batch_gradient_descent(data_matrix, 3, [2,3,4], 0.1 ,0.4, 2)

From data in epoch 0 and no_input 0 has error 0.03921915718610222
From data in epoch 0 and no_input 1 has error 0.039601462255480016
From data in epoch 0 and no_input 2 has error 0.2582390021867373
From data in epoch 0 and no_input 3 has error 0.25746230872564724
From data in epoch 0 and no_input 4 has error 0.2582625887763463
From data in epoch 0 and no_input 5 has error 0.039583455874482636
From data in epoch 0 and no_input 6 has error 0.25807160788338174
From data in epoch 0 and no_input 7 has error 0.03967472667402338
From data in epoch 0 and no_input 8 has error 0.25925642119467374
From data in epoch 0 and no_input 9 has error 0.2585884940683054
From data in epoch 0 and no_input 10 has error 0.2590555370943857
From data in epoch 0 and no_input 11 has error 0.25705703408741476
From data in epoch 0 and no_input 12 has error 0.2587675424659936
From data in epoch 0 and no_input 13 has error 0.03993389706493124
From data in epoch 1 and no_input 0 has error 0.03921915718610222
From data